# Import Libraries

In [1]:
import os 
import cv2
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from skimage.feature import hog
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,precision_score, recall_score,f1_score

# Load Data

In [2]:
annot = os.listdir('/content/drive/MyDrive/10. Toán cho KHMT/As 4/folio')
annot_id = list(range(len(annot)))
print(annot)
print(annot_id)

['croton', 'mulberry leaf', 'jackfruit', 'bitter orange', 'coeur demoiselle', 'barbados cherry', 'ashanti blood', 'betel', 'coffee', 'caricature plant']
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


In [ ]:
X_train = []
X_test = []
y_train = []
y_test = []

In [7]:
path = '/content/drive/MyDrive/10. Toán cho KHMT/As 4/folio'
for x in annot_id:
  x_path = os.path.join(path, annot[x])
  img_name = os.listdir(x_path)
  X = []
  y = []

  for img in img_name:
    image = cv2.imread(os.path.join(x_path,img),0)
    X.append(cv2.resize(image, (int(image.shape[1]*0.1), int(image.shape[0]*0.1)) ))
    y.append(x)

  X_train_sp, X_test_sp, y_train_sp, y_test_sp = train_test_split(X,y,test_size= 0.3)
  X_train.extend(X_train_sp)
  X_test.extend(X_test_sp)
  y_train.extend(y_train_sp)
  y_test.extend(y_test_sp)

# Extracting Feature HOG

## Convert to 1D

In [6]:
def convert_to_1D(image):
  return np.reshape(image, image.shape[0]*image.shape[1])

In [8]:
X_train_1D = []
for x in X_train:
  X_train_1D.append(convert_to_1D(x))

X_test_1D = []
for x in X_test:
  X_test_1D.append(convert_to_1D(x))

## HOG (Histogram of Oriented Gradients)

In [9]:
def HOG_calculate(image):
  return hog(image, orientations=9,pixels_per_cell=(4,4),
             cells_per_block=(2,2))

In [10]:
X_train_HOG = []
for x in X_train:
  X_train_HOG.append(HOG_calculate(x))

X_test_HOG = []
for x in X_test:
  X_test_HOG.append(HOG_calculate(x))

# Training Model

## Model 1D

In [11]:
model_1d = LogisticRegression(max_iter=10000, random_state=1,n_jobs=2)
model_1d.fit(X_train_1D,y_train) 

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=10000,
                   multi_class='auto', n_jobs=2, penalty='l2', random_state=1,
                   solver='lbfgs', tol=0.0001, verbose=0, warm_start=False)

## Model HOG

In [12]:
model_HOG = LogisticRegression(max_iter=10000,random_state=1,n_jobs=2)
model_HOG.fit(X_train_HOG,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=10000,
                   multi_class='auto', n_jobs=2, penalty='l2', random_state=1,
                   solver='lbfgs', tol=0.0001, verbose=0, warm_start=False)

# Evaluating

In [13]:
y_pred_1D = model_1d.predict(X_test_1D)
y_pred_hog = model_HOG.predict(X_test_HOG)

In [18]:
y_train_pred_1D = model_1d.predict(X_train_1D)
y_train_pred_HOG = model_HOG.predict(X_train_HOG)

In [15]:
print('Evaluating in Test')
arr1 = np.array([[accuracy_score(y_test, y_pred_1D), 
                precision_score(y_test, y_pred_1D, average='micro'), 
                recall_score(y_test, y_pred_1D, average='micro'), 
                f1_score(y_test, y_pred_1D, average='micro')]],
               dtype='float32')
arr2 = np.array([[accuracy_score(y_test, y_pred_hog), 
                 precision_score(y_test, y_pred_hog, average='micro'), 
                 recall_score(y_test, y_pred_hog, average='micro'), 
                 f1_score(y_test, y_pred_hog, average='micro')]],
               dtype='float32')
arr = np.concatenate((arr1, arr2), axis=0)

arr_df = pd.DataFrame(data=arr, index=['MODEL 1D', 'MODEL HOG'], 
                      columns=['ACCURACY', 'PRECISION', 'RECALL', 'F1'])
pd.options.display.float_format = '{:.3f}'.format
arr_df

Evaluating in Test


,ACCURACY,PRECISION,RECALL,F1
MODEL 1D,0.909,0.909,0.909,0.909
MODEL HOG,0.848,0.848,0.848,0.848


In [19]:
print('Evaluating in Train')
arr1 = np.array([[accuracy_score(y_train, y_train_pred_1D), 
                precision_score(y_train, y_train_pred_1D, average='micro'), 
                recall_score(y_train, y_train_pred_1D, average='micro'), 
                f1_score(y_train, y_train_pred_1D, average='micro')]],
               dtype='float32')
arr2 = np.array([[accuracy_score(y_train, y_train_pred_HOG), 
                 precision_score(y_train, y_train_pred_HOG, average='micro'), 
                 recall_score(y_train, y_train_pred_HOG, average='micro'), 
                 f1_score(y_train, y_train_pred_HOG, average='micro')]],
               dtype='float32')
arr = np.concatenate((arr1, arr2), axis=0)

arr_df = pd.DataFrame(data=arr, index=['MODEL 1D', 'MODEL HOG'], 
                      columns=['ACCURACY', 'PRECISION', 'RECALL', 'F1'])
pd.options.display.float_format = '{:.3f}'.format
arr_df

Evaluating in Train


,ACCURACY,PRECISION,RECALL,F1
MODEL 1D,1.000,1.000,1.000,1.000
MODEL HOG,1.000,1.000,1.000,1.000
